In [1]:
import numpy as np
import pickle
import torch
import time

from sklearn.metrics import r2_score, mean_absolute_error, explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split
import autosklearn.regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
plt.matplotlib.style.use("https://gist.githubusercontent.com/JonnyCBB/c464d302fefce4722fe6cf5f461114ea/raw/64a78942d3f7b4b5054902f2cee84213eaff872f/matplotlibrc")
cool_colors = ['#00BEFF', '#D4CA3A', '#FF6DAE', '#67E1B5', '#EBACFA', '#9E9E9E', '#F1988E', '#5DB15A', '#E28544', '#52B8AA']
cool_colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']

plt.rcParams.update({'font.size': 14})

The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The keymap.all_axes rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_path rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_args rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


load data from `prepare_Xy.ipynb`

In [2]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
print("shape of X:", np.shape(X))

y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

nb_COFs = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['nb_COFs']
print("# COFs:", nb_COFs)

nb_iterations = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['nb_iterations']
print("# iterations:", nb_iterations)

nb_runs = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['nb_runs']
print("# runs:", nb_runs)

# feature names
features = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['features']

shape of X: (500, 12)
shape of y: (500,)
# COFs: 500
# iterations: 100
# runs: 5


set this flag to determined whether the random forest uses a diverse training set (`True`) or a randomly selected training set (`False`).

In [3]:
diversify_training = True

In [4]:
"""
select a diverse training set (the rest is testing data)
"""
def diverse_train_test_split(X, train_size):
    ids_train = [np.random.randint(0, nb_COFs)] # initialize with one random point; pick others in a max diverse fashion
    # select remaining training points
    for j in range(train_size - 1):
        # for each point in data set, compute its min distance to training set
        distances_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
        assert np.shape(distances_to_train_set) == (len(ids_train), nb_COFs)
        min_distances_to_a_training_pt = np.min(distances_to_train_set, axis=0)
        assert np.size(min_distances_to_a_training_pt) == nb_COFs
        
        # acquire point with max min distance to train set (Furthest from train set)
        ids_train.append(np.argmax(min_distances_to_a_training_pt))
    assert np.size(np.unique(ids_train)) == train_size # gotta be unique
    
    # test data is wutever's left
    ids_test = [i for i in range(nb_COFs) if not i in ids_train]
    assert np.size(np.unique(ids_test)) == nb_COFs - train_size
    
    return np.array(ids_train), np.array(ids_test)

a random forest run.

In [5]:
def rf_run(nb_training_data, nb_acquire):
    if diversify_training:
        print("\tdiverse RF run")
    else:
        print("\tRF run")
    print("\teval budget", nb_training_data + nb_acquire, "=", nb_training_data, "training data and", nb_acquire, "acquired.")
    
    # test/train split
    if diversify_training:
        ids_train, ids_test = diverse_train_test_split(X, nb_training_data)
    else:
        ids_train, ids_test = train_test_split(np.arange(nb_COFs), train_size=nb_training_data)
    
    X_train = X[ids_train, :]
    y_train = y[ids_train]
    
    X_test  = X[ids_test, :]
    
    # train random forest on training data
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)

    # hv random forest make predictions on test data
    y_pred = rf.predict(X_test)

    # rank the test predictions
    ids_test_ranked = np.flip(np.argsort(y_pred))
    
    # acquire the COFs in the test set with highest predicted property
    ids_acquire = ids_test[ids_test_ranked[:nb_acquire]]

    # return the acquired COFs but also the trained COFs which count.
    ids_acquire_incld_training = np.concatenate((ids_acquire, ids_train))
    
    assert np.size(np.unique(ids_acquire_incld_training)) == nb_training_data + nb_acquire
    
    print("\tmax y acquired = ", np.max(y[ids_acquire_incld_training]))
    return ids_acquire_incld_training

In [6]:
rf_res = dict()
rf_res['nb_evals_budgets'] = [20 * i for i in range(1, int(nb_iterations/20 + 1))] + [250]
print("eval budgets: ", rf_res['nb_evals_budgets'])
rf_res['ids_acquired']     = [[] for b in rf_res['nb_evals_budgets']]
for b in range(len(rf_res['nb_evals_budgets'])):
    nb_evals_budget = rf_res['nb_evals_budgets'][b]
    print("budget for evals:", nb_evals_budget)
    # decide how to spend the evals budget here. say 50/50
    nb_training_data = nb_evals_budget // 2
    nb_acquire = nb_evals_budget // 2
    assert nb_training_data + nb_acquire == nb_evals_budget
    
    for r in range(nb_runs):
        print("\trun", r)
        ids_acquired = rf_run(nb_training_data, nb_acquire)
        rf_res['ids_acquired'][b].append(ids_acquired)

eval budgets:  [20, 40, 60, 80, 100, 250]
budget for evals: 20
	run 0
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  186.52243366599998
	run 1
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  180.412781897
	run 2
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  205.963467853
	run 3
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  174.497718622
	run 4
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  205.963467853
budget for evals: 40
	run 0
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  205.963467853
	run 1
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  205.963467853
	run 2
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  205.963467853
	run 3
	diverse RF run
	eval budget 40 = 20 tra

save results to file

In [7]:
if not diversify_training:
    with open('rf_results.pkl', 'wb') as file:
        pickle.dump(rf_res, file)
else:
    with open('rf_div_results.pkl', 'wb') as file:
        pickle.dump(rf_res, file)